<a href="https://colab.research.google.com/github/ufbfung/clinterm/blob/main/terminology_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Notebook will be used for demonstrating examples of how to use all the different pieces within the terminology package.

# Plan to use clinterm as term

# Setup environment

## Import relevant libraries
These will eventually go into a requirements.txt when deployed into pypi.

In [8]:
# Import relevant libraries
# These will eventually go into a requirements.txt once ready

import os # for working with directories
import requests # for working with apis
import json # for working with apis
import pandas as pd # for working with dataframes
import numpy as np # for working with arrays
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting
import plotly.express as px

## Mount google drive
This section is primarily for development where all the files associated with development will be hosted on a personal google drive for easier access.

In [9]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths
root_path = '/content/drive/My Drive/coding/clinterm'

# Change working directory to clinterm package
%cd {root_path}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/coding/clinterm


## Define helper functions
This section will eventually turn into a global utils package that can be used across all the terminologies, especially for managing api calls.

### Define helper functions as part of notebook for now until we are ready to export helperfunctions

In [29]:
def api_request(url, headers=None, params=None):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status() # Raise an exception for unsuccessful responses
        data = response.json() # if i want to convert to df earlier
        results = data['results'] # if i want to convert to df earlier
        df = pd.json_normalize(results)  # Flatten the nested JSON response
        return df # if i want to convert to df earlier
        # return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        return None

def retrieve_terminology_info(terminology, code):
    base_url = f'https://api.example.com/{terminology}/v1/search'
    query = f'{base_url}?code={code}'

    response_data = make_api_request(query)
    if response_data:
        # Process the response data for the specific terminology
        pass
    else:
        print(f"No data found for code: {code}")

def check_base_urls(base_urls):
    print("Checking the status of all APIs currently active in clinterm")
    print("------------------------------------------------------------")

    for base_url in base_urls:
        print(f"\nChecking API: {base_url}")
        response = requests.get(base_url)
        if response.status_code == 200:
            print("API is available.")
        else:
            print(f"API is not available. Status code: {response.status_code}")

### Export the helperfunctions into its own Python program
Will use this once we're ready to begin exporting

In [15]:
# Helper functions that will be used across apis
# Define them here for now and eventually move them into its own separate location

%%writefile helperfunctions.py
# Write your Python code here
def print_function():
  print("Hello, world2!")

Writing helperfunctions.py


In [18]:
import helperfunctions as hf

hf.print_function()

Hello, world2!


# Terminology development
This section will be used to develop the different APIs that will be used for working with the different terminologies

## Endpoint availability
Before working with these APIs, we're going to begin by checking the status of every API endpoint to ensure it hasn't changed and that they are all available

In [25]:
# Check all available APIs
# This section will eventually be stored as a variable, maybe environmental variable that will be referenced somewhere else
base_urls = [
    'https://api.fda.gov/drug/event.json',
    'https://api.fda.gov/drug/ndc.json',
    'https://fhir.loinc.org'
]

check_base_urls(base_urls)

Checking the status of all APIs currently active in clinterm
------------------------------------------------------------

Checking API: https://api.fda.gov/drug/event.json
API is available.

Checking API: https://api.fda.gov/drug/ndc.json
API is available.

Checking API: https://fhir.loinc.org
API is available.


## NDC
This will use the APIs from openFDA to retrieve information related to medications.

In [30]:
query = 'https://api.fda.gov/drug/ndc.json?search=dosage_form:%22LOTION%22&limit=1'
api_request(query)

,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,finished,packaging,listing_expiration_date,marketing_category,dosage_form,...,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,openfda.manufacturer_name,openfda.spl_set_id,openfda.is_original_packager,openfda.unii
0,78232-005,PAIN RELIEF,Clarity Labs LLC,GR Arthritis pain relief roll-on 1500 mg,"[{'name': 'CAMPHOR (SYNTHETIC)', 'strength': '...",True,"[{'package_ndc': '78232-005-03', 'description'...",20231231,OTC MONOGRAPH FINAL,LOTION,...,HUMAN OTC DRUG,[TOPICAL],20220405,78232-005_dbde562a-16e0-78a1-e053-2a95a90a2d93,part346,GR Arthritis pain relief roll-on 1500 mg,[Clarity Labs LLC],[dbde5519-5add-a2dd-e053-2a95a90ae80b],[True],"[5TJD82A1ET, L7T10EIP3A]"
